In [ ]:
"""
Purpose: To visualize the scores of different neurons
and figure out how we can improve the version 3 process
before the big run


"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
import datajoint_utils as du
minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-21 02:06:16,332 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-21 02:06:16,334 - settings - Setting database.user to celiib
INFO - 2021-04-21 02:06:16,335 - settings - Setting database.password to newceliipass
INFO - 2021-04-21 02:06:16,339 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-21 02:06:16,340 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-21 02:06:16,355 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-21 02:06:16,729 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-21 02:06:16,732 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-21 02:06:16,733 - settings - Setting database.user to celiib
INFO - 2021-04-21 02:06:16,734 - settings - Setting database.password to newceliipass
INFO - 2021-04-21 02:06:16,735 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-21 02:06:16,736 - settings - Setting database.user to celiib
INFO - 2021-04-21 02:06:16,737 - settings - Setting database.password to newceliipass
INFO - 2021-04-21 02:06:16,739 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-21 02:06:17,134 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-21 02:06:17,173 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-21 02:06:17,174 - settings - Setting database.user to celiib
INFO - 2021-04-21 02:06:17,175 - settings - Setting database.pa

In [4]:
from pathlib import Path
import numpy as np
import neuron_visualizations as nviz
import datajoint as dj

import proofreading_utils as pru
import classification_utils as clu

import neuron_utils as nru
import validation_utils as vu
import pandas as pd

In [5]:
import validation_utils as vu

In [10]:
decimation_version = 0
decimation_ratio = 0.25
process_version = 7

key_source = ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0")
                 & minnie.AutoProofreadValidationSegment()) 
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134988386682,0,0.25
864691135113167769,0,0.25
864691135212632448,0,0.25
864691135212690816,0,0.25
864691135233108569,0,0.25
864691135272164113,0,0.25
864691135348272855,0,0.25
864691135382947307,0,0.25
864691135393964789,0,0.25
864691135407289801,0,0.25


# Getting the Validation Statistics

In [ ]:
segment_ids = du.proofreading_stats_table(validation=True).fetch("segment_id")

validation_dfs = [vu.synapse_validation_df_single_neuron(k,
                                        remove_presyn_on_dendrite = False,
                        count_presyn_on_dendrite_as_correct = True,
                                                        include_extension_synapse=False) for k in segment_ids]

# Develop the Visualizations for the FP, FN, Tp, TN visualization function

In [ ]:
curr_validation_df = validation_dfs[6]

In [ ]:
du.plot_proofread_validation_neuron(
    #segment_id = 864691136370815112,
    validation_df=curr_validation_df,
    verbose = True,
    synapse_type = "presyn",
)

# Trying the new crossover resolve method

In [ ]:
segment_id = vu.validation_df_to_old_segment_id(curr_validation_df)
segment_id

In [ ]:
"""
Pseudocode: 

"""

neuron_objs,neuron_split_idxs = du.decomposition_with_spine_recalculation(segment_id)

In [ ]:
neuron_obj = neuron_objs[0]
neuron_obj.segment_id

In [ ]:
clu.axon_classification(neuron_obj)

In [ ]:
import proofreading_utils as pru
o_neuron,filter_dict = pru.proofread_neuron_class_predetermined(neuron_obj,
    perform_axon_classification=False,
    inh_exc_class="excitatory",
    plot_limb_branch_filter_with_disconnect_effect = True,

    high_fidelity_axon_on_excitatory = True,
    plot_final_filtered_neuron = False,

    #arguments for the axon high fidelity:
    plot_new_axon_limb_correspondence = True,
    plot_new_limb_object = False,
    plot_final_revised_axon_branch = False,

    verbose = False,
    verbose_outline = True)

In [ ]:
nviz.visualize_neuron(o_neuron,
                     limb_branch_dict="all")

In [ ]:
matched_branches = list(return_info[0])
match_branches_angle = list(return_info[1])
matched_branches,match_branches_angle



In [ ]:


return_info = xu.degree_1_max_edge_min_max_weight_graph(
    G = G,
    verbose = False,
    plot_winning_graph = False,
return_edge_info=True)

return_info

#xu.get_edges_with_weights(ret_G)

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()
G.add_weighted_edges_from([[1,2,10],[1,3,20],[1,4,50],[4,5,1]])
nx.draw(G,with_labels=True)

In [ ]:
xu.sum_of_edge_weights(G)

In [ ]:
xu.max_node_degree(G)

In [ ]:
import networkx_utils as xu
xu.all_subgraph_edges(G)

In [ ]:
G.score()